In [8]:
import os
from collections import defaultdict, Counter
import json

from tqdm import tqdm_notebook as tqdm

# project definitions 

In [9]:
LINUX_BASE = '/mnt/c/Users/mwhitmore/Desktop/Work/linux/'
LINUX_REPO_BASE = os.path.join(LINUX_BASE, 'repo')
SNAPSHOTS_BASE = os.path.join(LINUX_BASE, 'snapshots')
DATA_IN = os.path.join(LINUX_BASE, 'data_in')
os.makedirs(DATA_IN, exist_ok=True)

In [10]:
VERSIONS = {
    'v2.6.12-rc2':'2005-04-16 15:20:36 -0700',
    'v2.6.21' : '2007-04-25 20:08:32 -0700',
    'v2.6.31':'2009-09-09 15:13:59 -0700',
    'v3.0':'2011-07-21 19:17:23 -0700',    
    'v3.11':'2013-09-02 13:46:10 -0700',        
    'v4.0':'2015-04-12 15:12:50 -0700',            
    'v4.13':'2017-09-03 13:56:17 -0700',
    'v5.0':'2019-03-03 15:21:29 -0800'
}

In [11]:
def tool_file_path(version, tool):
    return os.path.join(DATA_IN, f'{version}.{tool}')

def snap_folder_path(version):
    return os.path.join(SNAPSHOTS_BASE, version)    

In [12]:
def run_wc(version):
    snap_folder = snap_folder_path(version)
    tool_file = tool_file_path(version, 'wc')
    
    cmd = f'find "{snap_folder}" -type f'
    cmd += r' -exec wc -l "{}" \;'
    cmd += f' > "{tool_file}"' 

    !{cmd}
    

In [13]:
def run_lineset(version):
    snap_folder = snap_folder_path(version)
    tool_file = tool_file_path(version, 'lineset')
    
    result = defaultdict(Counter)

    for root, subdirs, files in os.walk(snap_folder):
        for f in files:
            _, ext = os.path.splitext(f)
            ext = ext.lower()
            with open(os.path.join(snap_folder, root, f), encoding='latin-1') as fin:
                result[ext].update([l.strip() for l in fin])

    with open(tool_file, 'w') as fout:
        json.dump(result, fout, indent=1)

# Run WC

In [14]:
for version in tqdm(VERSIONS):
    print('generate profile data for version', version)
    run_wc(version)

generate profile data for version v2.6.12-rc2
generate profile data for version v2.6.21
generate profile data for version v2.6.31
generate profile data for version v3.0
generate profile data for version v3.11
generate profile data for version v4.0
generate profile data for version v4.13
generate profile data for version v5.0


# Run Lineset

In [15]:
for version in tqdm(VERSIONS):
    run_lineset(version)

In [16]:
# dependency, commit log, architecture tree